In [31]:
import pandas as pd
import os
import time
import logging
from datetime import date

In [32]:
# # 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)

In [33]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [34]:
folder_path = '../shopee_product_translation/'
# tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'tengus_tw_product_list.xlsx'))
tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'translated_product_2019_10_19.xlsx'))
tw_site_product_list = tw_site_product_list[tw_site_product_list['是否翻译'] == 'no']
# tw_site_product_list.head(5)

In [35]:
# tw_site_product_list.columns

In [36]:
# tw_site_product_list = tw_site_product_list[tw_site_product_list['产品id'] == 2496637042]

In [37]:
# top_cat_in_tw = tw_site_product_list[['分类ID', '产品id']].groupby('分类ID').agg({'产品id': 'nunique'}).reset_index().sort_values(by='产品id', ascending=False).reset_index().head(5)
# top_cat_in_tw

In [38]:
# top_cat_id = top_cat_in_tw['分类ID'][0]
# top_cat_id

In [39]:
output_excel_columns = [
                        '原有行',
                        '是否翻译',
                        '变种名称',
                        '变种属性名称一',
                        '变种属性名称二',
                        '变种属性值一',
                        '变种属性值二',
                        '*价格（必填）',
                        '*库存（必填）',
                        '*重量(kg)(必填）',
                        '*主图（URL）地址（必填）',
                        '附图1',
                        '附图2',
                        '附图3',
                        '附图4',
                        '附图5',
                        '附图6',
                        '附图7',
                        '附图8',
                        '变种图',
                        '长(cm)',
                        '宽(cm)',
                        '高(cm)',
                        '发货期',
                        '来源URL',
                        '尺码图'
]

In [40]:
tw_site_product_list_selected_columns = ['原有行', '是否翻译', '分类ID', '产品属性', 'Parent SKU', '产品标题', '产品描述', 'sku', '变种名称', '变种属性名称一',
       '变种属性名称二', '变种属性值一', '变种属性值二', '价格', '库存', '重量', '主图（URL）地址', '附图1',
       '附图2', '附图3', '附图4', '附图5', '附图6', '附图7', '附图8地址', '变种图', '长（cm）',
       '宽（cm）', '高（cm）', '发货期', '来源URL', '尺码图']

In [41]:
tw_site_product_list_with_selected_columns = tw_site_product_list[tw_site_product_list_selected_columns]
tw_site_product_list_with_selected_columns.head(5)

,原有行,是否翻译,分类ID,产品属性,Parent SKU,产品标题,产品描述,sku,变种名称,变种属性名称一,...,附图6,附图7,附图8地址,变种图,长（cm）,宽（cm）,高（cm）,发货期,来源URL,尺码图
0,5935,no,8315,"[{""mandatory"":true,""attribute_id"":9502,""attrib...",5.737442e+11,【限時特價】現貨 兒童糖果色打底褲 純棉長褲 男女童睡褲 女寶秋褲 長褲 兒童內搭褲 男童條...,款號：5440\n材質：100%純棉\n特點：吸濕透氣，柔軟舒適\n\n【貼心小叮嚀】 \n...,5440-顏色備註-80cm,顏色備註-80cm,Variation,...,https://s-cf-tw.shopeesz.com/file/e77e203a9980...,https://s-cf-tw.shopeesz.com/file/5d94bef9ace4...,https://s-cf-tw.shopeesz.com/file/925b5a2f71da...,NaN,18,13,5,2,https://detail.1688.com/offer/573744240242.htm...,NaN
1,5936,no,8315,"[{""mandatory"":true,""attribute_id"":9502,""attrib...",5.737442e+11,【限時特價】現貨 兒童糖果色打底褲 純棉長褲 男女童睡褲 女寶秋褲 長褲 兒童內搭褲 男童條...,款號：5440\n材質：100%純棉\n特點：吸濕透氣，柔軟舒適\n\n【貼心小叮嚀】 \n...,5440-顏色備註-90cm,顏色備註-90cm,Variation,...,https://s-cf-tw.shopeesz.com/file/e77e203a9980...,https://s-cf-tw.shopeesz.com/file/5d94bef9ace4...,https://s-cf-tw.shopeesz.com/file/925b5a2f71da...,NaN,18,13,5,2,https://detail.1688.com/offer/573744240242.htm...,NaN
2,5937,no,8315,"[{""mandatory"":true,""attribute_id"":9502,""attrib...",5.737442e+11,【限時特價】現貨 兒童糖果色打底褲 純棉長褲 男女童睡褲 女寶秋褲 長褲 兒童內搭褲 男童條...,款號：5440\n材質：100%純棉\n特點：吸濕透氣，柔軟舒適\n\n【貼心小叮嚀】 \n...,5440-顏色備註--100cm,顏色備註-100cm,Variation,...,https://s-cf-tw.shopeesz.com/file/e77e203a9980...,https://s-cf-tw.shopeesz.com/file/5d94bef9ace4...,https://s-cf-tw.shopeesz.com/file/925b5a2f71da...,NaN,18,13,5,2,https://detail.1688.com/offer/573744240242.htm...,NaN
3,5938,no,8315,"[{""mandatory"":true,""attribute_id"":9502,""attrib...",5.737442e+11,【限時特價】現貨 兒童糖果色打底褲 純棉長褲 男女童睡褲 女寶秋褲 長褲 兒童內搭褲 男童條...,款號：5440\n材質：100%純棉\n特點：吸濕透氣，柔軟舒適\n\n【貼心小叮嚀】 \n...,5440-顏色備註--110cm,顏色備註-110cm,Variation,...,https://s-cf-tw.shopeesz.com/file/e77e203a9980...,https://s-cf-tw.shopeesz.com/file/5d94bef9ace4...,https://s-cf-tw.shopeesz.com/file/925b5a2f71da...,NaN,18,13,5,2,https://detail.1688.com/offer/573744240242.htm...,NaN
4,5939,no,8315,"[{""mandatory"":true,""attribute_id"":9502,""attrib...",5.737442e+11,【限時特價】現貨 兒童糖果色打底褲 純棉長褲 男女童睡褲 女寶秋褲 長褲 兒童內搭褲 男童條...,款號：5440\n材質：100%純棉\n特點：吸濕透氣，柔軟舒適\n\n【貼心小叮嚀】 \n...,5440-顏色備註--120cm,顏色備註-120cm,Variation,...,https://s-cf-tw.shopeesz.com/file/e77e203a9980...,https://s-cf-tw.shopeesz.com/file/5d94bef9ace4...,https://s-cf-tw.shopeesz.com/file/925b5a2f71da...,NaN,18,13,5,2,https://detail.1688.com/offer/573744240242.htm...,NaN


In [42]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[tw_site_product_list_with_selected_columns['分类ID'] == top_cat_id].reset_index()
# tw_site_product_list_with_selected_columns

In [43]:
# 把1688的来源url拆出来，把productid填充为parent sku
tw_site_product_list_with_selected_columns['Parent SKU'] = tw_site_product_list_with_selected_columns['来源URL'].str.split("/", n = 4, expand = True)[4].str.split(".", n = 1, expand=True)[0]
# tw_site_product_list_with_selected_columns.head(5)

In [44]:
# remove emoji
import re

emoji_pattern = re.compile(
                u'['
                u'\U0001F300-\U0001F64F'
                u'\U0001F680-\U0001F6FF'
                u'\u2600-\u2B55'
                u'\u23cf'
                u'\u23e9'
                u'\u231a'
                u'\u3030'
                u'\ufe0f'
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u'\U00010000-\U0010ffff'
                u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                u'\U00002702-\U000027B0]+', flags=re.UNICODE)

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

remove_emoji('ins爆款 🥕胡蘿卜🥕寶寶針織竹節棉爬服 嬰兒兔裝 嬰兒連身衣 寶寶包屁衣連身衣 中間開襟包屁衣 寶寶連體衣哈衣爬服')

'ins爆款 胡蘿卜寶寶針織竹節棉爬服 嬰兒兔裝 嬰兒連身衣 寶寶包屁衣連身衣 中間開襟包屁衣 寶寶連體衣哈衣爬服'

In [45]:
tw_site_product_list_with_selected_columns['产品标题'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品标题']]
tw_site_product_list_with_selected_columns['产品描述'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品描述']]

In [46]:
tw_site_product_list_with_selected_columns['长（cm）'] = tw_site_product_list_with_selected_columns['长（cm）'].fillna(18)
tw_site_product_list_with_selected_columns['宽（cm）'] = tw_site_product_list_with_selected_columns['宽（cm）'].fillna(13)
tw_site_product_list_with_selected_columns['高（cm）'] = tw_site_product_list_with_selected_columns['高（cm）'].fillna(5)
tw_site_product_list_with_selected_columns['发货期'] = tw_site_product_list_with_selected_columns['发货期'].fillna(2)

In [47]:
translate_currency = 0.045
profit_rate = 1.2
tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * translate_currency / profit_rate

In [48]:
# tw_site_product_list_with_selected_columns.head(5)

In [49]:
from googletrans import Translator

In [50]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])
translator.translate('韩国', dest='ko').text

'한국'

In [51]:
# translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
# for translation in translations:
#     print(translation.origin, ' -> ', translation.text)

In [52]:
translate_columns = ['产品标题', '产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']
# translate_columns = ['产品标题']
# translate_columns = ['产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']

In [53]:
# 把空值填充为string格式
tw_site_product_list_with_selected_columns[translate_columns] = tw_site_product_list_with_selected_columns[translate_columns].fillna(value='不需翻译')

In [54]:
# 把variation_name做一些删减
variation_list = set(list(tw_site_product_list_with_selected_columns['变种属性名称一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性名称二'].unique()))
logging.info('variation list: ' + str(variation_list))

 2019-10-19 15:11:10,780 - INFO - variation list: {'not set', '尺碼', '款式', 'Variation'}


In [55]:
variation_rename = {'适合身高': '身高', '童袜尺码': '尺码', '不需翻译': 'not set'}
tw_site_product_list_with_selected_columns['变种属性名称一'] = tw_site_product_list_with_selected_columns['变种属性名称一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性名称二'] = tw_site_product_list_with_selected_columns['变种属性名称二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)

In [56]:
# 对variation_value做一些删减
# variation_value_list = set(list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()))
# logging.info('variation value list: ' + str(variation_value_list))

In [57]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[:5]
# # tw_site_product_list_with_selected_columns

In [58]:
today = date.today()
d1 = today.strftime("%Y_%m_%d")

In [59]:
# for index, row in tw_site_product_list_with_selected_columns.iterrows():
#     try:
#         list_to_translate = []
#         for i, value in enumerate(translate_columns):
#             list_to_translate.append(row[translate_columns[i]])
        
#         translate_language = 'en'
#         translator = Translator(service_urls=['translate.google.com'])
#         list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
#         # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
#         time.sleep(10)
        
#         i = 0
#         for translate_language in list_be_translated:
#             tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
#             i += 1
        
#         if (index + 1) % 5 == 0:
#             logging.info(str(index + 1) + ' rows have been translated.')
            
#         tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_' + d1 + '.xlsx'), index=False)
#     except Exception as err:
#         logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err))
#         pass

In [60]:
for index, row in tw_site_product_list_with_selected_columns.iterrows():
    translated = 0
    while translated == 0:
        try:
            list_to_translate = []
            for i, value in enumerate(translate_columns):
                list_to_translate.append(row[translate_columns[i]])

            translate_language = 'en'
            translator = Translator(service_urls=['translate.google.com'])
            list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
            # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
            time.sleep(10)

            i = 0
            for translate_language in list_be_translated:
                tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
                i += 1

            if (index + 1) % 5 == 0:
                logging.info(str(index + 1) + ' rows have been translated.')

            tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_' + d1 + '(1).xlsx'), index=False)
            translated = 1
        except Exception as err:
            logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err) + ', try again.')
            pass        

 2019-10-19 15:12:27,856 - INFO - 5 rows have been translated.
 2019-10-19 15:13:48,878 - INFO - 10 rows have been translated.
 2019-10-19 15:15:09,942 - INFO - 330 rows have been translated.
 2019-10-19 15:16:27,816 - INFO - 335 rows have been translated.
 2019-10-19 15:17:46,509 - INFO - 340 rows have been translated.
 2019-10-19 15:19:06,806 - INFO - 345 rows have been translated.
 2019-10-19 15:20:22,958 - INFO - 350 rows have been translated.
 2019-10-19 15:21:42,385 - INFO - 355 rows have been translated.
 2019-10-19 15:23:01,913 - INFO - 360 rows have been translated.
 2019-10-19 15:24:20,256 - INFO - 365 rows have been translated.
 2019-10-19 15:25:40,552 - INFO - 370 rows have been translated.
 2019-10-19 15:27:03,184 - INFO - 375 rows have been translated.
 2019-10-19 15:28:23,512 - INFO - 380 rows have been translated.
 2019-10-19 15:29:43,255 - INFO - 385 rows have been translated.
 2019-10-19 15:31:03,664 - INFO - 390 rows have been translated.
 2019-10-19 15:32:23,751 - I

In [61]:
# for translate_column in translate_columns:
#     # tw_site_product_list_with_selected_columns[translate_column] = tw_site_product_list_with_selected_columns[translate_column].fillna(0)
#     # tw_site_product_list_with_selected_columns[translate_column]  = [translator.translate(x, dest=translate_language).text 
#                                                                      # for x in tw_site_product_list_with_selected_columns[translate_column]]
#     for index, row in tw_site_product_list_with_selected_columns.iterrows():
#         try:
#             orginal_text = tw_site_product_list_with_selected_columns.at[index, translate_column]
#             tw_site_product_list_with_selected_columns.at[index, translate_column] = translator.translate(row[translate_column], dest=translate_language).text
#             # print(row[translate_column])
#             logging.info(translate_column + ' - ' + str(index + 1) + ' rows have been translated.')
#             # logging.info(orginal_text + ' -> ' + tw_site_product_list_with_selected_columns.at[index, translate_column])
#             time.sleep(5)
#         except:
#             logging.warning(translate_column + ' - ' + str(index + 1) + ' rows have not been translated.')
#             pass
        
#     # print(tw_site_product_list_with_selected_columns.head(5))
    
#     tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'tw_translated_product(1).xlsx'), index=False)
#     logging.info(str(translate_column) + ' is done.')